In [1]:
import pandas as pd
import polars as pl
import util
from pathlib import Path

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
transit_jobs_access = pd.read_csv(util.output_path / 'access/transit_jobs_access.csv', 
                                  usecols=['geography', 'value', 'geography_group'])
walk_bike_jobs_access = pd.read_csv(util.output_path / 'access/walk_bike_jobs_access.csv', 
                                  usecols=['geography_value', 'jobs_1_mile_walk', 'jobs_3_mile_bike', 'geography_group']).\
                                  rename(columns={'geography_value': 'geography'})

parcel_emp = util.get_parcels_urbansim_data(inc_geog=True)

# jobs access in equity geographies
equity_geogs = util.summary_config['equity_geogs']

## Jobs Accessible within 45 Minutes of Transit

In [3]:
def process_access_data(jobs_access):
    df_access = jobs_access.copy()
    # rename region
    df_access.loc[jobs_access['geography_group'] == 'region', 'geography'] = 'Region'
    # rename rgc
    df_access.loc[jobs_access['geography_group'] == 'rgc_binary', 'geography'] = ['Not in RGC', 'In RGC']
    # rename regional geography
    df_access.loc[jobs_access['geography_group'] == 'rg_proposed', 'geography_group'] = 'RegionalGeogName'

    equity_geogs = list(util.equity_geog_dict.keys())
    df_access_equity = df_access.loc[
        df_access['geography_group'].isin(equity_geogs)].copy()
    
    df_access_equity['geography'] = df_access_equity['geography'].map({"0.0": 'Below Regional Average', 
                                                                       "1.0": 'Above Regional Average', 
                                                                       "2.0": 'Higher Share of Equity Population'}
                                                                       )
    df_access_equity['geography_group'] = df_access_equity['geography_group'].map(util.equity_geog_dict)

    # df_access_equity_geogs['geography'] = df_access_equity_geogs['geography_group']
    # df['geography'] = "NOT in " + df['geography_group']

    # df_access_equity_geogs = pd.concat([df_access_equity_geogs, df], ignore_index=True)
    # df_access_equity_geogs['geography_group'] = 'Equity Geography'

    return df_access, df_access_equity


df_access_t, df_access_equity_t = process_access_data(transit_jobs_access)
df_access_bp, df_access_equity_bp = process_access_data(walk_bike_jobs_access)
tot_jobs = parcel_emp['emptot_p'].sum()

In [4]:
def job_access_geog(access_table, geog, format=True):
    df = access_table.loc[access_table['geography_group'].isin([geog, 'region'])].\
        rename(columns={'value': 'Jobs within 45-minute Transit Commute'}).\
        drop(columns=['geography_group']).\
        set_index('geography')
    
    df_jobs = parcel_emp.groupby(geog)['emptot_p'].sum()
    df_jobs['Region'] = tot_jobs
    df = pd.concat([df, df_jobs.rename('Total Jobs within Geography')], axis=1)
    df = df.loc[df.index != 'Outside Region'].copy()

    df['% All Jobs'] = df['Jobs within 45-minute Transit Commute'] / tot_jobs
    df['% of Jobs in Geography'] = df['Jobs within 45-minute Transit Commute'] / df['Total Jobs within Geography']

    if format:
        return df.style.format('{:,.1%}', subset=['% All Jobs','% of Jobs in Geography']).\
            format('{:,.0f}', subset=['Jobs within 45-minute Transit Commute','Total Jobs within Geography'])
    else:
        return df


In [5]:
df = job_access_geog(df_access_t,'CountyName')
# df = df[df.index != 'Outside Region']
df

,Jobs within 45-minute Transit Commute,Total Jobs within Geography,% All Jobs,% of Jobs in Geography
King,"421,636","2,048,436",13.3%,20.6%
Kitsap,"20,884","147,104",0.7%,14.2%
Pierce,"58,215","472,355",1.8%,12.3%
Snohomish,"110,105","493,145",3.5%,22.3%
Region,"259,614","3,161,040",8.2%,8.2%


In [6]:
df_rgc = job_access_geog(df_access_t,'rgc_binary')
df_rgc

,Jobs within 45-minute Transit Commute,Total Jobs within Geography,% All Jobs,% of Jobs in Geography
Region,"259,614","3,161,040",8.2%,8.2%
Not in RGC,"182,727","1,951,051",5.8%,9.4%
In RGC,"600,266","1,209,989",19.0%,49.6%


In [7]:
df = job_access_geog(df_access_t,'GrowthCenterName')

df

,Jobs within 45-minute Transit Commute,Total Jobs within Geography,% All Jobs,% of Jobs in Geography
Region,"259,614","3,161,040",8.2%,8.2%
Auburn,"174,716","9,693",5.5%,"1,802.5%"
Bellevue,"874,233","98,694",27.7%,885.8%
Bothell Canyon Park,"65,337","19,536",2.1%,334.4%
Bremerton,"69,412","24,223",2.2%,286.6%
Burien,"474,779","8,962",15.0%,"5,297.7%"
Everett,"218,324","53,778",6.9%,406.0%
Federal Way,"217,768","12,628",6.9%,"1,724.5%"
Greater Downtown Kirkland,"526,863","20,446",16.7%,"2,576.9%"
Issaquah,"256,619","15,474",8.1%,"1,658.4%"


In [8]:
job_access_geog(df_access_t,'RegionalGeogName')


,Jobs within 45-minute Transit Commute,Total Jobs within Geography,% All Jobs,% of Jobs in Geography
Region,"259,614","3,161,040",8.2%,8.2%
Cities and Towns,"29,799","142,611",0.9%,20.9%
Core Cities,"169,487",nan,5.4%,nan%
High Capacity Transit Communities,"121,154",nan,3.8%,nan%
Metropolitan Cities,"540,585",nan,17.1%,nan%
Rural Areas,"5,227",nan,0.2%,nan%
Urban Unincorporated Areas,"27,900",nan,0.9%,nan%


In [9]:
df = pd.DataFrame()
for col, label in util.equity_geog_dict.items():
    _df = job_access_geog(df_access_equity_t, label, format=False)
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df.rename(columns={'index': 'EFA Type'}, inplace=True)
# df
df.loc[df['EFA Type']!="Region"][['Group', 'EFA Type', 'Jobs within 45-minute Transit Commute',
    'Total Jobs within Geography', '% All Jobs', '% of Jobs in Geography']].style.\
        format('{:,.1%}', subset=['% All Jobs','% of Jobs in Geography']).\
        format('{:,.0f}', subset=['Jobs within 45-minute Transit Commute','Total Jobs within Geography'])

,Group,EFA Type,Jobs within 45-minute Transit Commute,Total Jobs within Geography,% All Jobs,% of Jobs in Geography
0,People of Color,Below Regional Average,"201,429","1,200,687",6.4%,16.8%
1,People of Color,Above Regional Average,"310,385","1,134,042",9.8%,27.4%
2,People of Color,Higher Share of Equity Population,"337,308","826,311",10.7%,40.8%
4,Income,Below Regional Average,"257,872","1,486,001",8.2%,17.4%
5,Income,Above Regional Average,"251,889","922,457",8.0%,27.3%
6,Income,Higher Share of Equity Population,"276,291","752,582",8.7%,36.7%
8,LEP,Below Regional Average,"238,977","1,643,280",7.6%,14.5%
9,LEP,Above Regional Average,"296,782","788,420",9.4%,37.6%
10,LEP,Higher Share of Equity Population,"283,081","729,340",9.0%,38.8%
12,Disability,Below Regional Average,"286,954","1,514,257",9.1%,19.0%


## Average Jobs Accessible within 1 Mile Walk and 3 Mile Bike
Note that this is not using the bike network, but is instead using the all-streets network.

Average accessible jobs are weighted averages based on parcel household population.

In [10]:
def bp_job_access_geog(access_table,geog):
    df = access_table.loc[access_table['geography_group'].isin(['region', geog])].\
        rename(columns={'jobs_1_mile_walk': 'Jobs within 1-mile Walk',
                        'jobs_3_mile_bike': 'Jobs within 3-mile Bike'}).\
        drop(columns=['geography_group']).\
        set_index('geography')

    df['% Total Jobs (1-mile walk)'] = df['Jobs within 1-mile Walk'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')
    df['% Total Jobs (3-mile bike)'] = df['Jobs within 3-mile Bike'].apply(lambda x: f'{x / tot_jobs * 100:,.1f}' + '%')

    return df

In [11]:
df = bp_job_access_geog(df_access_bp,'CountyName')
df = df[df.index != 'Outside Region']
df

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
King,"37,271","134,135",1.2%,4.2%
Kitsap,"2,746","14,528",0.1%,0.5%
Pierce,"8,283","37,911",0.3%,1.2%
Snohomish,"6,443","36,812",0.2%,1.2%
Region,"22,998","87,445",0.7%,2.8%


In [12]:
df_rgc = bp_job_access_geog(df_access_bp,'rgc_binary')
df = bp_job_access_geog(df_access_bp,'GrowthCenterName')

pd.concat([df_rgc, df.loc[~df.index.isin(['Region','Not in RGC'])]], axis=0)

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"22,998","87,445",0.7%,2.8%
Not in RGC,"4,676","49,794",0.1%,1.6%
In RGC,"104,176","254,260",3.3%,8.0%
Auburn,"19,999","58,805",0.6%,1.9%
Bellevue,"105,531","180,124",3.3%,5.7%
Bothell Canyon Park,"13,092","34,742",0.4%,1.1%
Bremerton,"20,449","48,459",0.6%,1.5%
Burien,"10,870","22,675",0.3%,0.7%
Everett,"48,673","91,292",1.5%,2.9%


In [13]:
bp_job_access_geog(df_access_bp,'RegionalGeogName')

,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
geography,,,,
Region,"22,998","87,445",0.7%,2.8%
Cities and Towns,"1,383","12,707",0.0%,0.4%
Core Cities,"8,355","49,234",0.3%,1.6%
High Capacity Transit Communities,"2,793","26,296",0.1%,0.8%
Metropolitan Cities,"57,573","194,889",1.8%,6.2%
Rural Areas,243,"3,539",0.0%,0.1%
Urban Unincorporated Areas,685,"10,604",0.0%,0.3%


In [14]:
df = pd.DataFrame()
for col, label in util.equity_geog_dict.items():
    _df = bp_job_access_geog(df_access_equity_bp, label)
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df.rename(columns={'geography': 'EFA Type'}, inplace=True)
df[['Group', 'EFA Type', 'Jobs within 1-mile Walk',	'Jobs within 3-mile Bike',	'% Total Jobs (1-mile walk)','% Total Jobs (3-mile bike)']]

,Group,EFA Type,Jobs within 1-mile Walk,Jobs within 3-mile Bike,% Total Jobs (1-mile walk),% Total Jobs (3-mile bike)
0,People of Color,Below Regional Average,"9,267","57,941",0.3%,1.8%
1,People of Color,Above Regional Average,"37,997","120,237",1.2%,3.8%
2,People of Color,Higher Share of Equity Population,"36,058","114,525",1.1%,3.6%
3,Income,Below Regional Average,"20,136","82,624",0.6%,2.6%
4,Income,Above Regional Average,"25,381","89,441",0.8%,2.8%
5,Income,Higher Share of Equity Population,"28,396","99,479",0.9%,3.1%
6,LEP,Below Regional Average,"21,019","85,945",0.7%,2.7%
7,LEP,Above Regional Average,"31,701","95,949",1.0%,3.0%
8,LEP,Higher Share of Equity Population,"18,531","81,576",0.6%,2.6%
9,Disability,Below Regional Average,"20,802","89,249",0.7%,2.8%


## Intersection Density

In [15]:
buffered_parcels = pl.read_csv(util.output_path / 'landuse/buffered_parcels.txt', 
                               separator=' ',
                               columns=['parcelid','nodes3_2','nodes4_2','hh_p']).to_pandas()


list_cols = ['ParcelID','CountyName','GrowthCenterName','rg_proposed'] + equity_geogs
parcel_geog = util.get_parcel_geog()[list_cols]

In [16]:
df_intersection = buffered_parcels.merge(parcel_geog, left_on='parcelid', right_on='ParcelID')

# Total intersections within 1/2 mile buffer
df_intersection['intersections_wt'] = (df_intersection['nodes3_2'] + df_intersection['nodes4_2']) * df_intersection['hh_p']

In [17]:
def intersection_density(geog, map=False):
    df = df_intersection.groupby(geog)[['intersections_wt', 'hh_p']].sum().reset_index()
    df['Intersections'] = df['intersections_wt']/df['hh_p']

    if map:
        df[geog] = df[geog].astype('int').map({0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}
                                )
    
    return df[[geog] + ['Intersections']]

In [18]:
df = intersection_density('CountyName')
df = df[df['CountyName']!='Outside Region']
df

,CountyName,Intersections
0,King,171
1,Kitsap,57
3,Pierce,90
4,Snohomish,81


In [19]:
intersection_density('GrowthCenterName')

,GrowthCenterName,Intersections
0,Auburn,193
1,Bellevue,271
2,Bothell Canyon Park,64
3,Bremerton,167
4,Burien,175
5,Everett,157
6,Federal Way,138
7,Greater Downtown Kirkland,164
8,Issaquah,99
9,Kent,213


In [20]:
intersection_density('rg_proposed')

,rg_proposed,Intersections
0,Cities and Towns,64
1,Core Cities,111
2,High Capacity Transit Communities,87
3,Metropolitan Cities,216
4,Rural Areas,20
5,Urban Unincorporated Areas,51


In [21]:
df = pd.DataFrame()
for col, label in util.equity_geog_dict.items():
    _df = intersection_density(col).rename(columns={col: "Group"})
    _df['Group'] = label
    df = pd.concat([df, _df])
df = df.reset_index()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })
df[['Group', 'EFA Type', 'Intersections']]

,Group,EFA Type,Intersections
0,People of Color,Below Regional Average,105
1,People of Color,Above Regional Average,155
2,People of Color,Higher Share of Equity Population,152
3,Income,Below Regional Average,120
4,Income,Above Regional Average,132
5,Income,Higher Share of Equity Population,154
6,LEP,Below Regional Average,124
7,LEP,Above Regional Average,141
8,LEP,Higher Share of Equity Population,133
9,Disability,Below Regional Average,129
